In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc

In [ ]:
browser = uc.Chrome()

In [ ]:
url_grande = "https://www.idealista.com/venta-viviendas/zaragoza-zaragoza/"

In [ ]:
browser.get(url_grande)

In [ ]:
browser.implicitly_wait(10)


In [ ]:
browser.find_element("xpath",'//*[@id="didomi-notice-agree-button"]').click()

In [ ]:
html = browser.page_source

In [ ]:
# Parse the HTML
soup = bs(html, 'lxml')

# Find all articles
articles = soup.find('main', {'class': 'listing-items'}).find_all('article')

# Extract 'data-element-id' from each article
id_muebles = [article.get('data-element-id') for article in articles]

# Filter out None values
id_muebles = [muebles for muebles in id_muebles if muebles is not None]

print(id_muebles)


In [ ]:
x = 1
ids = []

while True:
    url = f'https://www.idealista.com/venta-viviendas/zaragoza-zaragoza/pagina-{x}.htm'
    browser.get(url)
    time.sleep(random.randint(10, 12))
    
    # Fix the xpath of the cookies button
    try:
        browser.find_element("xpath", '//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass

    html = browser.page_source
    soup = bs(html, 'lxml')

    # Check that elements exist before trying to extract the page number
    try:
        pagina_actual = int(soup.find('main', {'class': 'listing-items'})
                            .find('div', {'class': 'pagination'})
                            .find('li', {'class': 'selected'}).text)
    except AttributeError:
        print("Pagination could not be found.")
        break

    if x == pagina_actual:
        articles = soup.find('main', {'class': 'listing-items'}).find_all('article')
    else:
        break

    x = x + 1

    for article in articles:
        # Make sure the correct attribute is 'data-adid' or 'data-element-id'
        id_muebles = article.get('data-element-id')  # or article.get('data-adid') if that's the right one
        ids.append(id_muebles)
        time.sleep(random.randint(1, 3))

    # Filter out None values from ids
    ids = [muebles for muebles in ids if muebles is not None]

print(ids)


In [ ]:
ids_casas = pd.DataFrame(ids)
ids_casas.colums = ['id']
ids_casas.rename(columns={0: 'id'}, inplace=True)

In [ ]:
ids_casas.to_csv('ids_casas.csv', index = False)